# Intro to Spatial Data and Projections/Coordinate Systems

In [ ]:
import os
import geopandas as gpd
import contextily as cx
import folium
data_pth = "../Data/"

### Using Geopandas Datasets
More info: https://geopandas.org/en/stable/docs/user_guide/mapping.html

In [ ]:
# when geopandas is installed, some data gets installed as well, like country boundaries
countries = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
# this is spatial data (the Geometry column) but it also has non-spatial attributes, one record for each shape
countries.head(3)

In [ ]:
# the crs is the coordinate reference system which includes coordinate system, projection, and other geospatial information
# it is defined by an epsg code (more info at https://epsg.io/)

# let's look at the crs to make sure it is defined
countries.crs

In [ ]:
# now we can plot this data as a static image
countries.plot(figsize=(15,10))

In [ ]:
# we can use the interactive explore method (folium behind the scenes) to get an interactive interface
# Interactive mapping https://geopandas.org/en/stable/docs/user_guide/interactive_mapping.html

countries.explore(tiles='CartoDB positron')

In [ ]:
# geopandas also comes with city data, these are capital cities

capitals = gpd.read_file(gpd.datasets.get_path('naturalearth_cities'))

In [ ]:
# this data also has spatial and non-spatial attributes, one row for each shape

capitals.head(3)

In [ ]:
# let's double check the crs

capitals.crs

In [ ]:
# then explore the data

capitals.explore(tiles='CartoDB positron')

In [ ]:
# if we want to view the countires and cities together, its a good idea to verify that they are both in the same crs first

capitals.crs == countries.crs

In [ ]:
# to view both layers, set the base layer to m (map object), then pass the map object to any other layers

m = countries.explore(tiles='CartoDB positron') # define the countries as a map object, m
capitals.explore(m=m, # pass the map object
                 color='purple') # set the color to something other than the blue default for better visibility

In [ ]:
# choropleth map - a map in which areas are shaded according to an attribute of interest

# let's color the countries by their population

m = countries.explore(
     tiles='CartoDB positron',
     cmap='copper_r',
     column='pop_est',  # make choropleth based on "pop_est" column
     scheme='naturalbreaks',  # the natural breaks scheme - designed to determine the best arrangement of values into different classes
     legend=True, # show legend
     k=5, # use 5 bins
     legend_kwds=dict(colorbar=False), # do not use colorbar, instead group the colors
     name='Countries' # name of the layer in the layer control
)

# add the cities on top as a red circle
capitals.explore(
     m=m, # pass the map object
     color='purple', # use red color on all points
     marker_kwds=dict(radius=2, fill=True), # make marker radius 2px with fill
     name='Cities' # name of the layer in the layer control
)

folium.LayerControl().add_to(m)  # use folium to add layer control

m

### Add Data From Another Source

In [ ]:
# GeoPandas city data is the capital cities, so let's load in a larger cities dataset
# https://www.naturalearthdata.com/downloads/10m-cultural-vectors/10m-populated-places/

cities = gpd.read_file(os.path.join(data_pth, "ne_10m_populated_places.shp"))

In [ ]:
# let's look at the first few rows of the table

cities.head(3)

In [ ]:
# let's keep only cities with a population > 20,000

cities = cities[cities['RANK_MAX'] > 9]

In [ ]:
# add the countries with the simple default styling
m = countries.explore(
     tiles='CartoDB positron',
     name='Countries'
)

# then color the cities by population
cities.explore(
     m=m, # pass the map object
     column='POP_MAX', # color by the POP_MAX column
     cmap='copper_r', # color scheme
     scheme='naturalbreaks', # use natural breaks binning scheme
     k = 5, # use 5 bins
     marker_kwds=dict(radius=2, fill=True), # make marker radius 2px with fill
     legend_kwds=dict(colorbar=False),  # do not use colorbar, instead group the colors
     tooltip=['NAME', 'POP_MAX'],# show the 'NAME' and 'POP_MAX' columns in the tooltip
     name='Cities' # name of the layer in the layer control
)

folium.LayerControl().add_to(m)  # use folium to add layer control

m # display the map

### Projections

#### #1 Reproject to use a certain basemap

In [ ]:
# Use the Contextily library to use different basemap tiles on static maps
# https://contextily.readthedocs.io/en/latest/index.html
# to use these basemap tiles, reproject the data into Popular Visualisation Pseudo-Mercator, which is epsg 3857

countries_wm = countries.to_crs(epsg=3857)
cities_wm = cities.to_crs(epsg=3857)

# since Antarctica is at the southern pole, this projection does not work well with it, so we remove it
# this is not a requirement but it makes the map look nicer

countries_wm = countries_wm[countries.name!='Antarctica']

In [ ]:
# and create a static plot of the countries with the with the city dataset

base = countries_wm.plot(figsize=(15,10), alpha = 0.75, facecolor='none', edgecolor = 'blue')
cities_wm.plot(ax=base, color='brown', markersize=3)
cx.add_basemap(base)

#### #2 Reproject to perform measurements

In [ ]:
# you may get a warning if you attempt measurements using a projected crs (like latitude/longitude)
# so, a projected coordinate system must be used to perform calculations
# WGS 84 / World Equidistant Cylindrical is one option if mapping the world

countries_wec = countries.to_crs(epsg=4087)
cities_wec = cities.to_crs(epsg=4087)

In [ ]:
# plot the countries and cities with the new projection to see the difference

base = countries_wec.plot(figsize=(15,10), alpha = 0.75, facecolor='none', edgecolor = 'blue')
cities_wec.plot(ax=base, color='brown', markersize=3)

In [ ]:
# now we are working with meters rather than degrees

countries_wec.crs

In [ ]:
# get the area calculations for each area, they will be in metres as shown above

countries_wec.area

#### #3 Reproject to better view a certain area

In [ ]:
# if mapping a smaller area, it may be better to choose a projection optimized for that area
# to look at Antarctica specifically, separate it out into its own shape

antarctica = countries[countries.name=='Antarctica']

In [ ]:
# reproject it to a crs that works well for Antarctica
# WGS 84 / Antarctic Polar Stereographic / EPSG 3031

antarctica = antarctica.to_crs(epsg=3031)

In [ ]:
# this is a much more accurate depiction of antarctica
antarctica.plot(figsize=(15,10))

#### Projections could lead to unexpected results

In [ ]:
# note that if the countries and cities are set to this projection

countries_polar = countries.to_crs(epsg=3031)
cities_polar = cities.to_crs(countries_polar.crs)

In [ ]:
# it will look like this, so be intentional with the choice of projection

base = countries_polar.plot(figsize=(15,10))
cities_polar.plot(ax=base, color='purple', markersize=10)